In [3]:
import torch
import torch.nn as nn
import tqdm
import numpy as np
import torch.optim as optim
import torchvision
from torchvision.utils import make_grid, save_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
torch.cuda.get_device_name()

'Tesla T4'

In [5]:
class Critic(nn.Module):
    def __init__(self, img_channels, features_d):
        super().__init__()
        self.c = nn.Sequential(nn.Conv2d(img_channels, features_d, kernel_size=4, stride=2, padding=1), 
                                nn.LeakyReLU(0.2), 

                                self.conv_block(features_d, features_d * 2, 4, 2, 1), 
                                self.conv_block(features_d * 2, features_d * 4, 4, 2, 1), 
                                self.conv_block(features_d * 4, features_d * 8, 4, 2, 1), 

                                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0))
        
    def conv_block(self, in_channels, out_channels, kernel_size, stride, padding):
         return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False), 
                              nn.InstanceNorm2d(out_channels, affine=True), 
                              nn.LeakyReLU(0.2))

    def forward(self, x):
        return self.c(x)

In [6]:
class Generator(nn.Module):
    def __init__(self, noise_channels, img_channels, features_g):
        super().__init__()
        self.g = nn.Sequential(self.conv_block(noise_channels, features_g * 16, 4, 1, 0), 
                               self.conv_block(features_g * 16, features_g * 8, 4, 2, 1),
                               self.conv_block(features_g * 8, features_g * 4, 4, 2, 1), 
                               self.conv_block(features_g * 4, features_g * 2, 4, 2, 1),
                               
                               nn.ConvTranspose2d(features_g * 2, img_channels, kernel_size=4, stride=2, padding=1),
                               nn.Tanh())
        
    def conv_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False), 
                             nn.BatchNorm2d(out_channels), 
                             nn.ReLU())
    
    def forward(self, x):
        return self.g(x)

In [7]:
def init_parameters(net):
    for n in net.modules():
        if isinstance(n, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(n.weight.data, 0.0, 0.02)

In [8]:
def gradient_penalty(C, real, fake, device):
    bs, c, h, w = real.shape
    alpha = torch.rand((bs, 1, 1, 1)).repeat(1, c, h, w).to(device)
    juxtaposition = alpha * real + (1 - alpha) * fake
    scores = C(juxtaposition)

    gradient = torch.autograd.grad(outputs=scores, inputs=juxtaposition, grad_outputs=torch.ones_like(scores),
                                   create_graph=True, retain_graph=True)[0]

    gradient = gradient.view(gradient.shape[0], -1)
    grad_norm = gradient.norm(2, dim=1)
    gp = torch.mean((grad_norm - 1) ** 2)
    return gp

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lr = 1e-4
batch_size = 64
img_size = 64
img_channels = 3
noise_dim = 100
epochs = 500
features_c = 64
features_g = 64
critic_iter = 5
lambda_gp = 10

In [10]:
device

device(type='cuda')

In [11]:
transform = transforms.Compose([transforms.Resize((img_size, img_size)),
                                 transforms.ToTensor(), 
                                 transforms.Normalize([0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)])])

In [12]:
dataset = datasets.ImageFolder(root="faces/", transform=transform)

In [13]:
len(dataset)

15167

In [14]:
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [15]:
len(dataloader)

237

In [16]:
G = Generator(noise_dim, img_channels, features_g).to(device)
C = Critic(img_channels, features_c).to(device)
init_parameters(G)
init_parameters(C)

In [17]:
opt_G = optim.Adam(G.parameters(), lr, betas=(0.0, 0.9))
opt_C = optim.Adam(C.parameters(), lr, betas=(0.0, 0.9))

In [18]:
fixed_noise = torch.randn(64, noise_dim, 1, 1).to(device)

In [19]:
def save_fake_images(epoch_num):
    with torch.no_grad():
        fake = G(fixed_noise)
    img_grid = make_grid(fake, normalize=True)
    file_name = "fake_images-{0:0=4d}.png".format(epoch_num)
    save_image(fake, file_name, nrow=8)
    print("")
    print("Image Saved!")
    print("")

In [20]:
for real, _ in dataloader:
    img_grid = make_grid(real[:64], normalize=True)
    save_image(img_grid, "real_images.png", nrow=8)
    break

In [21]:
save_fake_images(0)


Image Saved!



In [ ]:
for epoch in range(epochs):
    batch_losses_C = []
    batch_losses_G = []
    for real, _ in tqdm.tqdm(dataloader, total=len(dataloader)):
        real = real.to(device)
        curr_batch_size = real.shape[0]

        for _ in range(critic_iter):
            noise = torch.randn(curr_batch_size, noise_dim, 1, 1).to(device)
            fake = G(noise)
            C_real = C(real).reshape(-1)
            C_fake = C(fake).reshape(-1)
            gp = gradient_penalty(C, fake, real, device)
            loss_C = -(torch.mean(C_real) - torch.mean(C_fake)) + lambda_gp * gp

            C.zero_grad()
            loss_C.backward(retain_graph=True)
            opt_C.step()
            batch_losses_C.append(loss_C.item())
        
        output = C(fake).reshape(-1)
        loss_G = -torch.mean(output)

        G.zero_grad()
        loss_G.backward()
        opt_G.step()
        batch_losses_G.append(loss_G.item())

    print(f" Epoch: {epoch} | Loss C: {np.round(sum(batch_losses_C)/len(batch_losses_C), 4)} | Loss G: {np.round(sum(batch_losses_G)/len(batch_losses_G), 4)}")
    if epoch % 10 == 0:
        save_fake_images(epoch)

  0%|          | 0/237 [00:00<?, ?it/s]

 Epoch: 0 | Loss C: -84.4116 | Loss G: 115.1006

Image Saved!



  0%|          | 0/237 [00:00<?, ?it/s]

 Epoch: 1 | Loss C: -47.3321 | Loss G: 141.1016


  0%|          | 0/237 [00:00<?, ?it/s]

...

  0%|          | 0/237 [00:00<?, ?it/s]

 Epoch: 71 | Loss C: -7.8312 | Loss G: 167.0011


  0%|          | 0/237 [00:00<?, ?it/s]

 Epoch: 72 | Loss C: -7.9153 | Loss G: 164.9164


 30%|██▉       | 70/237 [01:26<03:25,  1.23s/it]